# Technical Audit of Sentience Institute Study

### Purpose
* Evaluate the accuracy of the figures in this study
* Evaluate the methodology for reasonableness
* Evaluate whether the conclusions of this study match the data
* Consider whether this dataset is useful for further research purposes

### Data Description
* Population: US Adults
* Sample: 1,094 people, census-balanced based on age, sex, region, ethnicity, and income.
    * *Note: which census? How does Ipsos do census-balancing?*
    * Excludes individuals who failed the awareness check - 46.3% of people passed.
        * I wonder whether selecting based on attention biases the results?
    
### Main Findings
1. **54%** of US adults say they are “currently trying to **consume fewer animal-based foods** (meat, dairy, and/or eggs) **and more plant-based foods** (fruits, grains, beans, and/or vegetables).”
2. **97%** of US adults agree “Whether to eat animals or be vegetarian is a **personal choice**, and nobody has the right to tell me which one they think I should do.”
3. **49%** of US adults support a **ban on factory farming**
4. **47%** support a **ban on slaughterhouses**
5. **33%** support a **ban on animal farming**
6. **58%** of US adults think “most farmed animals are **treated well**”
7. **75%** of US adults say they usually buy animal products “**from animals that are treated humanely**”

In [30]:
import numpy as np
import pandas as pd
import seaborn as sb

from openpyxl import load_workbook
from openpyxl.utils import get_column_interval

In [35]:
import os
import re
from collections import Counter

In [7]:
data_dir = "data"
input_data = os.path.join(data_dir, "Animal Farming Attitudes Survey Data.xlsx")

def get_excel_range(filename, sheet_name, sheet_range):
    """
    FN suggested here: https://stackoverflow.com/questions/43327975/how-can-i-
    read-a-rangea5b10-and-place-these-values-into-a-dataframe-using-o
    """
    wb = load_workbook(filename=filename, read_only=True)
    ws = wb[sheet_name]
    
    sheet_range = re.sub("\s", "", sheet_range)
    ws_range = ws[sheet_range]
    header = [cell.value for cell in ws_range[0]]
    rows = [ [cell.value for cell in row] for row in ws_range[1:] ]
    
    return pd.DataFrame(rows, columns=header)

#### 1. Clean Raw Dataset

In [12]:
raw = get_excel_range(input_data, "Raw", "A1:AH1095")
raw.head()

,Run,Program Version,Time Started,Minutes Spent,Points,numberOfQuestions,increment,progress,Position,otx_id_1,...,I support a ban on slaughterhouses. (a20sbox),I support a ban on animal farming. (8mjexdf),"Suppose you were given $10 and allowed to donate any amount of it to an effective non-profit organization that works to help farmed animals, keeping the rest for yourself. How much of this $10 would you donate? (sbzmhrb)","Suppose a public demonstration against the problems of factory farming occurred near where you live and your friend asked you to come demonstrate with her. If this demonstration fit into your schedule, how likely would you be to join and help demonstrate? (yrd9ug1)","When these foods are the same price as animal-based foods, people should eat more of these foods and fewer animal-based foods. (5yfiuvk)","When these foods are the same price as animal-based foods, I would prefer to eat more of these foods and fewer animal-based foods. (g9j5zwx)","When these foods are the same price as animal-based foods, people should eat more of these foods and fewer animal-based foods. (nawdq1n)","When these foods are the same price as animal-based foods, I would prefer to eat more of these foods and fewer animal-based foods. (88inyp8)","Most farmed animals are treated well. For example, the animals are given enough space and kept in good health. (1uaomy1)","The animal-based foods I purchase (meat, dairy, and/or eggs) usually come from animals that are treated humanely. For example, the animals are given enough space and kept in good health. (drd18km)"
0,2891666.0,8.0,2017-10-06 23:53:40,3.97,0.0,18.0,0.055556,1.0,arutyo6,501854929.0,...,Disagree,Disagree,10,Somewhat likely,Somewhat disagree,Somewhat disagree,Strongly disagree,Strongly disagree,Somewhat agree,No opinion
1,2891675.0,8.0,2017-10-07 00:01:52,7.02,0.0,18.0,0.055556,1.0,arutyo6,501854985.0,...,Strongly disagree,Strongly disagree,5,Very unlikely,Agree,Strongly agree,Strongly agree,Strongly agree,Agree,No opinion
2,2891670.0,8.0,2017-10-06 23:59:43,1.52,0.0,18.0,0.055556,1.0,arutyo6,501855036.0,...,Somewhat agree,Somewhat disagree,3,Very likely,Somewhat agree,Somewhat agree,Somewhat disagree,Disagree,Strongly disagree,Somewhat disagree
3,2891681.0,8.0,2017-10-07 00:07:28,3.62,0.0,18.0,0.055556,1.0,arutyo6,501855307.0,...,No opinion,Strongly disagree,10,Somewhat likely,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat disagree,Somewhat disagree
4,2891716.0,8.0,2017-10-07 00:19:02,4.50,0.0,18.0,0.055556,1.0,arutyo6,501855695.0,...,Disagree,Disagree,0,Very unlikely,Disagree,Strongly disagree,Disagree,Strongly disagree,Agree,Agree


In [13]:
# rename column names for readability
raw.columns

Index(['Run', 'Program Version', 'Time Started', 'Minutes Spent', 'Points',
       'numberOfQuestions', 'increment', 'progress', 'Position', 'otx_id_1',
       'otx_id_2', 'Age', 'Gender', 'Region', 'Ethnicity', 'Education',
       'Income',
       'People should consume fewer animal-based foods (meat, dairy, and/or eggs) and more plant-based foods (fruits, grains, beans, and/or vegetables). (jaocxhv)',
       'I am currently trying to consume fewer animal-based foods (meat, dairy, and/or eggs) and more plant-based foods (fruits, grains, beans, and/or vegetables). (8xrlozv)',
       'I have some discomfort with the way animals are used in the food industry. (ev1wy8h)',
       'Farmed animals have roughly the same ability to feel pain and discomfort as humans. (hiia3w0)',
       'Whether to eat animals or be vegetarian is a personal choice, and nobody has the right to tell me which one they think I should do. (saaqnbp)',
       'The factory farming of animals is one of the most importan

In [17]:
# 1. rename question vars for ease of use. Save the text of the questions in a dictionary somewhere.
questions = [
    'People should consume fewer animal-based foods (meat, dairy, and/or eggs) and more plant-based foods (fruits, grains, beans, and/or vegetables). (jaocxhv)',
    'I am currently trying to consume fewer animal-based foods (meat, dairy, and/or eggs) and more plant-based foods (fruits, grains, beans, and/or vegetables). (8xrlozv)',
    'I have some discomfort with the way animals are used in the food industry. (ev1wy8h)',
    'Farmed animals have roughly the same ability to feel pain and discomfort as humans. (hiia3w0)',
    'Whether to eat animals or be vegetarian is a personal choice, and nobody has the right to tell me which one they think I should do. (saaqnbp)',
    'The factory farming of animals is one of the most important social issues in the world today. /(A factory farm is a large industrialized farm, especially one on which a large number of livestock are raised indoors in conditions intended to maximize production at minimal cost.)/ (r1z9y5o)',
    'I support a ban on the factory farming of animals. (zc3ae6)',
    'I support a ban on slaughterhouses. (a20sbox)',
    'I support a ban on animal farming. (8mjexdf)',
    'Suppose you were given $10 and allowed to donate any amount of it to an effective non-profit organization that works to help farmed animals, keeping the rest for yourself. How much of this $10 would you donate? (sbzmhrb)',
    'Suppose a public demonstration against the problems of factory farming occurred near where you live and your friend asked you to come demonstrate with her. If this demonstration fit into your schedule, how likely would you be to join and help demonstrate? (yrd9ug1)',
    'When these foods are the same price as animal-based foods, people should eat more of these foods and fewer animal-based foods. (5yfiuvk)',
    'When these foods are the same price as animal-based foods, I would prefer to eat more of these foods and fewer animal-based foods. (g9j5zwx)',
    'When these foods are the same price as animal-based foods, people should eat more of these foods and fewer animal-based foods. (nawdq1n)',
    'When these foods are the same price as animal-based foods, I would prefer to eat more of these foods and fewer animal-based foods. (88inyp8)',
    'Most farmed animals are treated well. For example, the animals are given enough space and kept in good health. (1uaomy1)',
    'The animal-based foods I purchase (meat, dairy, and/or eggs) usually come from animals that are treated humanely. For example, the animals are given enough space and kept in good health. (drd18km)'
]

format_dict = { text: "q{}".format(i + 1) for i, text in enumerate(questions) }
labels = { "q{}".format(i + 1): text for i, text in enumerate(questions) }

raw = raw.rename(index=str, columns=format_dict)
raw.columns

Index(['Run', 'Program Version', 'Time Started', 'Minutes Spent', 'Points',
       'numberOfQuestions', 'increment', 'progress', 'Position', 'otx_id_1',
       'otx_id_2', 'Age', 'Gender', 'Region', 'Ethnicity', 'Education',
       'Income', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10',
       'q11', 'q12', 'q13', 'q14', 'q15', 'q16', 'q17'],
      dtype='object')

In [23]:
# 2. Convert all vars to snake case
def make_snakecase(text):
    remove_spaces = re.sub("\s+", "_", text)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', remove_spaces).lower()

raw.columns = map(make_snakecase, raw.columns)
raw.columns

Index(['run', 'program_version', 'time_started', 'minutes_spent', 'points',
       'number_of_questions', 'increment', 'progress', 'position', 'otx_id_1',
       'otx_id_2', 'age', 'gender', 'region', 'ethnicity', 'education',
       'income', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10',
       'q11', 'q12', 'q13', 'q14', 'q15', 'q16', 'q17'],
      dtype='object')

In [24]:
key = get_excel_range(input_data, "Key", "A1:E54")
key.head()

,Question Name,Question Text,Data Type,Answer Name,Answer Text
0,AGE,what is your age?,Numeric Text,,
1,GENDER,what is your gender?,Single Punch,A1,male
2,GENDER,what is your gender?,Single Punch,A2,female
3,REGION,dummy question for zip-region4,Single Punch,A1,northeast
4,REGION,dummy question for zip-region4,Single Punch,A2,midwest


In [28]:
questions = list(map(make_snakecase, key["Question Name"]))
values = key["Answer Name"]
text = key["Answer Text"]
unique_key = zip(questions, values)
key_dict = dict(zip(unique_key, text))
list(key_dict.items())[:10]

[(('age', ''), ''),
 (('gender', 'A1'), 'male'),
 (('gender', 'A2'), 'female'),
 (('region', 'A1'), 'northeast'),
 (('region', 'A2'), 'midwest'),
 (('region', 'A3'), 'south'),
 (('region', 'A4'), 'west'),
 (('ethnicity', 'A1'), 'white or caucasian (not hispanic or latino)'),
 (('ethnicity', 'A2'), 'black or african-american (not hispanic or latino)'),
 (('ethnicity', 'A3'), 'asian/pacific islander')]

In [29]:
raw.head()

,run,program_version,time_started,minutes_spent,points,number_of_questions,increment,progress,position,otx_id_1,...,q8,q9,q10,q11,q12,q13,q14,q15,q16,q17
0,2891666.0,8.0,2017-10-06 23:53:40,3.97,0.0,18.0,0.055556,1.0,arutyo6,501854929.0,...,Disagree,Disagree,10,Somewhat likely,Somewhat disagree,Somewhat disagree,Strongly disagree,Strongly disagree,Somewhat agree,No opinion
1,2891675.0,8.0,2017-10-07 00:01:52,7.02,0.0,18.0,0.055556,1.0,arutyo6,501854985.0,...,Strongly disagree,Strongly disagree,5,Very unlikely,Agree,Strongly agree,Strongly agree,Strongly agree,Agree,No opinion
2,2891670.0,8.0,2017-10-06 23:59:43,1.52,0.0,18.0,0.055556,1.0,arutyo6,501855036.0,...,Somewhat agree,Somewhat disagree,3,Very likely,Somewhat agree,Somewhat agree,Somewhat disagree,Disagree,Strongly disagree,Somewhat disagree
3,2891681.0,8.0,2017-10-07 00:07:28,3.62,0.0,18.0,0.055556,1.0,arutyo6,501855307.0,...,No opinion,Strongly disagree,10,Somewhat likely,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat agree,Somewhat disagree,Somewhat disagree
4,2891716.0,8.0,2017-10-07 00:19:02,4.50,0.0,18.0,0.055556,1.0,arutyo6,501855695.0,...,Disagree,Disagree,0,Very unlikely,Disagree,Strongly disagree,Disagree,Strongly disagree,Agree,Agree


In [50]:
def get_col_freq(df=None, colname=None, series_obj=None):
    series_obj = series_obj if series_obj is not None else df[colname]
    return Counter(series_obj).most_common()

get_col_freq(df=raw, colname="points")

[(0.0, 1094)]

In [38]:
get_col_freq(df=raw, colname="number_of_questions")

[(18.0, 1094)]

In [39]:
get_col_freq(df=raw, colname="progress")

[(1.0, 1094)]

In [40]:
raw.columns

Index(['run', 'program_version', 'time_started', 'minutes_spent', 'points',
       'number_of_questions', 'increment', 'progress', 'position', 'otx_id_1',
       'otx_id_2', 'age', 'gender', 'region', 'ethnicity', 'education',
       'income', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10',
       'q11', 'q12', 'q13', 'q14', 'q15', 'q16', 'q17'],
      dtype='object')

In [41]:
get_col_freq(df=raw, colname="run")

[(2891666.0, 1),
 (2891675.0, 1),
 (2891670.0, 1),
 (2891681.0, 1),
 (2891716.0, 1),
 (2891745.0, 1),
 (2891710.0, 1),
 (2891757.0, 1),
 (2891769.0, 1),
 (2891771.0, 1),
 (2891783.0, 1),
 (2891810.0, 1),
 (2891859.0, 1),
 (2891856.0, 1),
 (2891866.0, 1),
 (2891860.0, 1),
 (2891865.0, 1),
 (2891872.0, 1),
 (2891873.0, 1),
 (2891884.0, 1),
 (2891892.0, 1),
 (2891901.0, 1),
 (2891897.0, 1),
 (2891947.0, 1),
 (2891957.0, 1),
 (2891989.0, 1),
 (2891978.0, 1),
 (2891979.0, 1),
 (2891980.0, 1),
 (2891983.0, 1),
 (2891997.0, 1),
 (2891988.0, 1),
 (2891996.0, 1),
 (2892010.0, 1),
 (2891995.0, 1),
 (2892006.0, 1),
 (2892005.0, 1),
 (2892007.0, 1),
 (2892016.0, 1),
 (2892019.0, 1),
 (2892023.0, 1),
 (2892025.0, 1),
 (2892027.0, 1),
 (2892029.0, 1),
 (2892031.0, 1),
 (2892036.0, 1),
 (2892037.0, 1),
 (2892039.0, 1),
 (2892042.0, 1),
 (2892051.0, 1),
 (2892052.0, 1),
 (2892060.0, 1),
 (2892050.0, 1),
 (2892055.0, 1),
 (2892054.0, 1),
 (2892062.0, 1),
 (2892061.0, 1),
 (2892064.0, 1),
 (2892066.0, 1

In [53]:
sorted(get_col_freq(series_obj=raw.minutes_spent.apply(lambda x: round(x, 0))))

[(0.0, 5),
 (1.0, 27),
 (2.0, 229),
 (3.0, 303),
 (4.0, 230),
 (5.0, 124),
 (6.0, 69),
 (7.0, 36),
 (8.0, 26),
 (9.0, 9),
 (10.0, 9),
 (11.0, 3),
 (12.0, 3),
 (13.0, 2),
 (14.0, 2),
 (16.0, 2),
 (17.0, 1),
 (18.0, 3),
 (21.0, 1),
 (25.0, 1),
 (26.0, 1),
 (27.0, 1),
 (30.0, 1),
 (34.0, 2),
 (39.0, 1),
 (41.0, 1),
 (64.0, 1),
 (91.0, 1)]